In [5]:
df1

,시가,고가,저가,종가,거래량,등락률,EMA_short,EMA_long,MACD,MACD_Signal,...,종가_minus_SMA_60,종가_minus_SMA_120,종가_minus_SMA_240,종가_minus_EMA_5,종가_minus_EMA_20,종가_minus_EMA_60,종가_minus_EMA_120,종가_minus_EMA_240,next_day_return,target
날짜,,,,,,,,,,,,,,,,,,,,,
2022-02-14,4825,4830,4810,4825,42053,0.000000,4742.884569,4729.290035,13.594534,-13.030989,...,53.750000,6.083333,86.812500,31.763700,96.244086,64.552240,45.788917,60.996956,-0.414508,-1
2022-02-15,4820,4845,4705,4805,43098,-0.414508,4752.440789,4734.898181,17.542608,-6.916269,...,35.500000,-13.041667,64.270833,7.842467,68.982745,43.091510,25.362654,40.656733,0.312175,1
2022-02-16,4805,4825,4805,4820,37372,0.312175,4762.834514,4741.202019,21.632495,-1.206516,...,51.916667,2.625000,76.750000,15.228311,75.984388,56.186871,39.695503,55.194851,0.207469,-1
2022-02-17,4820,4840,4810,4830,42436,0.207469,4773.167666,4747.779647,25.388018,4.112391,...,63.250000,12.875000,84.229167,16.818874,77.795399,64.016809,48.874090,64.653815,0.310559,1
2022-02-18,4830,4845,4820,4845,16569,0.310559,4784.218794,4754.981155,29.237639,9.137440,...,79.333333,27.916667,96.583333,21.212583,83.957742,76.426094,62.818319,78.992788,-0.103199,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-13,4290,4320,4285,4295,27559,-0.116279,4258.257582,4179.423072,78.834510,64.293208,...,223.166667,158.375000,107.229167,-11.799604,88.965763,164.093883,156.268926,102.421429,-0.116414,-1
2024-03-14,4300,4300,4270,4290,22399,-0.116414,4263.141031,4187.613955,75.527075,66.539981,...,215.583333,151.291667,101.541667,-11.199736,75.969023,153.877690,148.768613,96.612953,0.000000,-1
2024-03-15,4320,4320,4285,4290,31128,0.000000,4267.273180,4195.198107,72.075073,67.647000,...,213.583333,149.166667,100.895833,-7.466491,68.733878,148.832520,146.309628,95.811185,-0.582751,-1


In [2]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# SK하이닉스 종목코드
ticker = "448740"

# 시작 날짜와 종료 날짜 설정
start_date = "2020-03-20"
end_date = "2024-03-20"
# 주어진 기간 동안의 일별 거래량 정보 가져오기
df = stock.get_market_ohlcv_by_date(fromdate=start_date, todate=end_date, ticker=ticker)


def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """MACD 및 MACD 신호 계산"""
    df['EMA_short'] = df['종가'].ewm(span=short_window, adjust=False).mean()
    df['EMA_long'] = df['종가'].ewm(span=long_window, adjust=False).mean()
    df['MACD'] = df['EMA_short'] - df['EMA_long']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    return df

def calculate_rsi(df, window=14, signal_window=9):
    """RSI 및 RSI 신호 계산"""
    delta = df['종가'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # RSI 신호선 추가
    df['RSI_Signal'] = df['RSI'].ewm(span=signal_window, adjust=False).mean()
    return df

# MACD 및 MACD 신호 계산
df = calculate_macd(df)

df = calculate_rsi(df)

# '종가' 컬럼이 포함된 DataFrame을 가정합니다. 예를 들어, df라고 합시다.
# df = pd.read_csv('path_to_your_data.csv') # 데이터 파일을 로드하는 예시

# 지정된 기간에 대한 SMA 계산
periods = [5, 20, 60, 120, 240]
for period in periods:
    df[f'SMA_{period}'] = df['종가'].rolling(window=period).mean()

# 지정된 기간에 대한 EMA 계산
for period in periods:
    df[f'EMA_{period}'] = df['종가'].ewm(span=period, adjust=False).mean()

for i in range(len(periods)):
    for j in range(i + 1, len(periods)):
        smaller_period = periods[i]
        larger_period = periods[j]
        df[f'SMA_{smaller_period}_minus_SMA_{larger_period}'] = df[f'SMA_{smaller_period}'] - df[f'SMA_{larger_period}']

# EMA 간의 차이 계산
for i in range(len(periods)):
    for j in range(i + 1, len(periods)):
        smaller_period = periods[i]
        larger_period = periods[j]
        df[f'EMA_{smaller_period}_minus_EMA_{larger_period}'] = df[f'EMA_{smaller_period}'] - df[f'EMA_{larger_period}']

for column in ['시가', '고가', '저가','거래량','MACD','MACD_Signal','RSI','RSI_Signal']:
    df[f'{column}_등락률'] = df[column].pct_change() * 100

# SMA 및 EMA의 전 거래일 대비 등락률 계산 및 DataFrame에 추가
periods = [5, 20, 60, 120, 240]

# SMA 등락률 계산 및 추가
for period in periods:
    df[f'SMA_{period}_등락률'] = df[f'SMA_{period}'].pct_change() * 100

# EMA 등락률 계산 및 추가
for period in periods:
    df[f'EMA_{period}_등락률'] = df[f'EMA_{period}'].pct_change() * 100

# 'MACD'가 0 이상일 때 1, 아니면 0을 할당
df['MACD_Binary'] = (df['MACD'] >= 0).astype(int)

# 'MACD - MACD_Signal'이 0 이상일 때 1, 아니면 0을 할당
df['MACD_minus_Signal_Binary'] = ((df['MACD'] - df['MACD_Signal']) >= 0).astype(int)

# 'RSI - RSI_Signal'이 0 이상일 때 1, 아니면 0을 할당
df['RSI_minus_Signal_Binary'] = ((df['RSI'] - df['RSI_Signal']) >= 0).astype(int)

# 시가 대비 종가 등락률 컬럼 추가
df['시가_대비_종가_등락률'] = ((df['종가'] - df['시가']) / df['시가']) * 100
df['시가_대비_저가_등락률'] = ((df['저가'] - df['시가']) / df['시가']) * 100
df['시가_대비_고가_등락률'] = ((df['고가'] - df['시가']) / df['시가']) * 100
df['저가_대비_종가_등락률'] = ((df['종가'] - df['저가']) / df['저가']) * 100
df['저가_대비_고가_등락률'] = ((df['고가'] - df['저가']) / df['저가']) * 100
df['고가_대비_종가_등락률'] = ((df['종가'] - df['고가']) / df['고가']) * 100
# 종가 - SMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
for period in [5, 20, 60, 120, 240]:
    df[f'종가_minus_SMA_{period}'] = df['종가'] - df[f'SMA_{period}']

# 종가 - EMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
for period in [5, 20, 60, 120, 240]:
    df[f'종가_minus_EMA_{period}'] = df['종가'] - df[f'EMA_{period}']


# csv_file_path = "C:/apps/h1/이스트소프트_data.csv"  # 저장할 파일 경로 및 이름 설정
# df.to_csv(csv_file_path, encoding='utf-8-sig')

#print(f"Data saved to {csv_file_path}")

# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = df['등락률'].shift(-1)
df['target'] = df['next_day_return'].apply(lambda x: 2 if x > 0.25 else 1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df1= df.iloc[300:, :]
X = df.drop(['next_day_return','target'], axis=1)
y = df['target']
# # 무한대 값 NaN으로 변환
# X.replace([np.inf, -np.inf], np.nan, inplace=True)

# # NaN 값이 포함된 행 제거
# X.dropna(inplace=True)

# # 대상 변수 y도 동일한 행 제거를 위해 X와 인덱스를 맞춥니다.
# y = y.reindex(X.index)

# Calculating correlations for Forward Selection
correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
correlations.columns = ['Feature', 'Correlation']

# Performing T-tests
t_tests = {column: ttest_ind(X[column][y == 1], X[column][y == -1], nan_policy='omit') for column in X.columns}
t_tests_sorted = sorted(t_tests.items(), key=lambda x: x[1].pvalue)
t_tests_df = pd.DataFrame(t_tests_sorted, columns=['Feature', 'T-test'])
t_tests_df['T-test'] = t_tests_df['T-test'].apply(lambda x: x.pvalue)  # Only keep p-value for simplicity

# Lasso and Ridge regression
lasso = LassoCV().fit(X, y)
ridge = RidgeCV().fit(X, y)

lasso_importance = np.abs(lasso.coef_)
ridge_importance = np.abs(ridge.coef_)

features_lasso_sorted = sorted(zip(X.columns, lasso_importance), key=lambda x: x[1], reverse=True)
features_ridge_sorted = sorted(zip(X.columns, ridge_importance), key=lambda x: x[1], reverse=True)

features_lasso_df = pd.DataFrame(features_lasso_sorted, columns=['Feature', 'Lasso Importance'])
features_ridge_df = pd.DataFrame(features_ridge_sorted, columns=['Feature', 'Ridge Importance'])

# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# # Combining all the data into a single dataframe without merging by feature
# final_df = pd.concat([correlations, t_tests_df.drop('Feature', axis=1), features_lasso_df.drop('Feature', axis=1), 
#                       features_ridge_df.drop('Feature', axis=1), features_rf_df.drop('Feature', axis=1)], axis=1)

# csv_file_path = "C:/apps/h1/NH프라임리츠_importance_data.csv"  # 저장할 파일 경로 및 이름 설정
# final_df.to_csv(csv_file_path, encoding='utf-8-sig')
top_8_features = correlations.sort_values(by='Correlation', ascending=False).head(8)['Feature']

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)
X_top8 = X[top_8_features]
y = df['target'] # y 값을 0과 1로 조정
X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=42)


# 모델 초기화
models = {
    'SVC': SVC(),
    'LogisticRegression': LogisticRegression(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier()
}

# 점수를 저장할 딕셔너리
scores = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1_Score': []
}

# 각 모델에 대해 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    scores['Model'].append(model_name)
    scores['Accuracy'].append(accuracy_score(y_test, y_pred))
    scores['Precision'].append(precision_score(y_test, y_pred, average='macro'))
    scores['Recall'].append(recall_score(y_test, y_pred, average='macro'))
    scores['F1_Score'].append(f1_score(y_test, y_pred, average='macro'))

# 점수 딕셔너리를 데이터프레임으로 변환
scores_df = pd.DataFrame(scores)
scores_df

c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.466e-03, tolerance: 9.333e-05
  model = cd_fast.enet_coordinate_descent(


,Model,Accuracy,Precision,Recall,F1_Score
0,SVC,1.0,1.0,1.0,1.0
1,LogisticRegression,1.0,1.0,1.0,1.0
2,DecisionTreeClassifier,1.0,1.0,1.0,1.0
3,RandomForestClassifier,1.0,1.0,1.0,1.0


In [4]:
correlations.head(10)

,Feature,Correlation
0,MACD_minus_Signal_Binary,0.685994
1,종가_minus_EMA_20,0.572227
2,EMA_20_등락률,0.567498
3,종가_minus_SMA_20,0.492355
4,EMA_5_minus_EMA_20,0.491351
5,EMA_60_등락률,0.477690
6,종가_minus_EMA_5,0.475794
7,SMA_5_등락률,0.475733
8,EMA_5_등락률,0.474390
9,종가_minus_EMA_60,0.467756


In [3]:
df

,시가,고가,저가,종가,거래량,등락률,EMA_short,EMA_long,MACD,MACD_Signal,...,종가_minus_SMA_60,종가_minus_SMA_120,종가_minus_SMA_240,종가_minus_EMA_5,종가_minus_EMA_20,종가_minus_EMA_60,종가_minus_EMA_120,종가_minus_EMA_240,next_day_return,target
날짜,,,,,,,,,,,,,,,,,,,,,
2024-02-22,9650,9680,9640,9670,21865,0.311203,9600.292071,9565.755065,34.537006,23.195052,...,154.666667,123.000000,12.375000,34.389529,92.616379,128.158159,102.188824,20.593560,0.103413,1
2024-02-23,9680,9700,9670,9680,22026,0.103413,9612.554829,9574.217653,38.337176,26.223476,...,161.666667,133.166667,23.416667,29.593019,92.843390,133.628384,110.334463,30.339672,0.103306,1
2024-02-26,9680,9690,9670,9690,8848,0.103306,9624.469471,9582.794123,41.675348,29.313851,...,168.333333,143.250000,34.583333,26.395346,93.048782,138.919256,118.345463,40.004903,-0.103199,1
2024-02-27,9690,9690,9660,9680,4064,-0.103199,9633.012629,9589.994558,43.018071,32.054695,...,155.166667,133.416667,25.833333,10.930231,75.139374,124.692395,106.554629,29.755899,0.103306,1
2024-02-28,9670,9700,9670,9690,12433,0.103306,9641.779917,9597.402369,44.377548,34.519265,...,162.000000,143.583333,36.791667,13.953487,77.030862,130.276251,114.628107,39.425975,0.103199,1
2024-02-29,9690,9700,9670,9700,5994,0.103199,9650.736853,9605.002193,45.734659,36.762344,...,168.333333,153.416667,47.791667,15.968991,78.742209,135.677030,122.568138,49.015801,0.206186,1
2024-03-04,9690,9720,9670,9720,11906,0.206186,9661.392722,9613.520549,47.872172,38.984310,...,184.666667,173.166667,68.583333,23.979328,89.338189,150.572865,140.211640,68.443055,0.000000,1
2024-03-05,9720,9740,9680,9720,10488,0.000000,9670.409226,9621.407916,49.001310,40.987710,...,180.833333,173.083333,69.333333,15.986218,80.829790,145.636050,137.894092,67.875063,0.000000,1
2024-03-06,9710,9720,9680,9720,5726,0.000000,9678.038576,9628.711033,49.327542,42.655676,...,177.166667,173.000000,69.833333,10.657479,73.131715,140.861097,135.614851,67.311785,0.000000,1
